In [1]:
import os
import sys
import urllib.request
import json
from multiprocessing import Pool, cpu_count, Process
from tqdm.auto import tqdm
from functools import partial
import base64
import requests
from urllib import parse
import uuid
import time
from random import randint
import pandas as pd

class Translator_Nologin:
    client_id = "SswNccvb51DzGQHARCG5"
    client_secret = "GU5cvgz94P"
    url = "https://openapi.naver.com/v1/papago/n2mt"
    def papao_get_translation_en_to_ko(text):
        encText = urllib.parse.quote(text)
        data = "source=en&target=ko&text=" + encText    
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request, data=data.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            json_object = json.loads(response_body.decode('utf-8'))
            return json_object['message']['result']['translatedText']
        else:
            raise Exception

    def papao_get_translation_ko_to_en(text):
        encText = urllib.parse.quote(text)
        data = "source=ko&target=en&text=" + encText
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request, data=data.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            json_object = json.loads(response_body.decode('utf-8'))
            return json_object['message']['result']['translatedText']
        else:
            raise Exception
        
class Translator:
    """
    Main Translator Class
    """

    def __init__(self, headers=None):        
        self.headers = headers or {
            'device-type': 'pc',
            'origin': 'https://papago.naver.com',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'ko',
            'authority': 'papago.naver.com',
            'pragma': 'no-cache',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
            'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'accept': 'application/json',
            'cache-control': 'no-cache',
            'x-apigw-partnerid': 'papago',
            'referer': 'https://papago.naver.com/',
            'sec-fetch-mode': 'cors',
        }
        self.generation_uuid = [str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4())]
        
    def translate(self, query, source='en', target='ko', verbose=False):
        data = {
                'deviceId': self.generation_uuid[randint(0, 3)],
                'locale': target,
                'dict': True,
                'dictDisplay': 30,
                'honorific': False,
                'instant': False,
                'paging': False,
                'source': source,
                'target': target,
                'text': query
        }
        print(json.dumps(data))
        data = 'data={}'.format(parse.quote(json.dumps(data)))
        print(data)
        response = requests.post('https://papago.naver.com/apis/n2mt/translate', headers=self.headers, data=data)
        print(response.json())
        if not verbose:
            return [response.json()['translatedText'], query]
        return response.json()

    def bulk_translate(self, queries, source='en', target='ko', workers=None, verbose=False):
        with Pool(workers) as pool:
            result = list(tqdm(pool.imap(
                func=partial(self.translate, source=source, target=target, verbose=verbose),
                iterable=queries
            ), total=len(queries)))
            pool.close()
            pool.join()
            return result

In [ ]:
if __name__ == '__main__':
    obj = Translator()
    with open('v2_OpenEnded_mscoco_train2014_questions.json', 'r') as json_file:
        v2_OpenEnded_mscoco_train2014_questions = json.load(json_file)
    questions = pd.DataFrame(v2_OpenEnded_mscoco_train2014_questions['questions'])
    dup_questions = questions.drop_duplicates(['question'], keep='last')
    result = obj.bulk_translate(dup_questions['question'][:10], workers=8)
    with open('v2_OpenEnded_mscoco_train2014_questions_output.json', 'w') as json_file:
        json.dump(result, json_file)